# Ejercicios

Esta sesión consta de un ejercicio mínimo y de varias ampliaciones que no son obligatorias.

## Ejercicio mínimo (5 puntos)

Usando el notebook de explicación construye un modelo para el dataset de neutralización y súbelo a HuggingFace. Incluye en la siguiente celda el enlace a tu modelo en HuggingFace y guarda los cambios en GitHub.  


Enlace al modelo: https://huggingface.co/jumava/mbart-neutralization

## Instalando librerías

Por defecto, el entorno de Google Colab no tiene instaladas las librerías de [HuggingFace](https://huggingface.co/), por lo que vamos a hacer en primer lugar es instalar las librerías: [Transformers](https://huggingface.co/docs/transformers/index), [Datasets](https://huggingface.co/docs/datasets/index), y [Evaluate](https://huggingface.co/docs/evaluate/index).

In [ ]:
!pip install datasets evaluate transformers[sentencepiece] sacrebleu accelerate -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

A continuación nos conectamos al hub de huggingface, lo que nos permitirá subir nuestros modelos a este entorno. Al ejecutar la siguiente celda aparecerá un widget en el cual tendremos que copiar el token generado en el primer paso y pulsar en el botón login.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset

Para este ejemplo vamos a utilizar el [Spanish Gender Neutralization
dataset](https://huggingface.co/datasets/hackathon-pln-es/neutral-es) que contiene un dataset para traducir de español "normal" a español "inclusivo".

Comenzamos descargando el dataset.

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("hackathon-pln-es/neutral-es")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.38k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/177k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3513 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/96 [00:00<?, ? examples/s]

Veamos el contenido de este dataset.

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['gendered', 'neutral'],
        num_rows: 3513
    })
    test: Dataset({
        features: ['gendered', 'neutral'],
        num_rows: 96
    })
})

Podemos ver que tenemos un objeto DatasetDict que puede verse como un diccionario. Dicho diccionario contiene un atributo `train` y un atributo `test`, por lo que en este caso ya disponemos de la división para entrenar nuestro modelo. Vamos a ver alguna de las frases del dataset, para lo que tenemos que acceder al atributo `train`.

In [ ]:
dataset['train']

Dataset({
    features: ['gendered', 'neutral'],
    num_rows: 3513
})

Con el anterior comando vemos que tenemos un Dataset con dos columnas: `gendered`, y `neutral`. Si queremos ver el contenido del dataset, lo podemos transformar a formato pandas y verlo como una tabla. Vemos a continuación que cada fila de nuestro dataset contiene una frase y una frase alineada que está neutralizada.

In [ ]:
dataset['train'].to_pandas()

,gendered,neutral
0,Se han incorporado al centro dos docentes para...,Se han incorporado al centro dos docentes para...
1,Tenemos que cubrir las carencias que presenta ...,Tenemos que cubrir las carencias que presenta ...
2,Habla con tu médico para averiguar si necesit...,Habla con tu médico para averiguar si necesit...
3,"Mientras tanto , el espectador comenzaba a im...","Mientras tanto , la audiencia comenzaba a imp..."
4,Los intermediarios del acuerdo se ocuparán tam...,El equipo intermediadior del acuerdo se ocupar...
...,...,...
3508,Es importante cubrir las carencias que tiene e...,Es importante cubrir las carencias que tiene e...
3509,la empleada doméstica ha terminado su trabajo ...,el servicio doméstico ha terminado su trabajo ...
3510,Las páginas web podrán publicar haciendo const...,Las páginas web podrán publicar haciendo const...
3511,realizamos entrevistas con los encargados de ...,realizamos entrevistas con el personal respon...


Para poder entrenar un modelo con este dataset es necesario tokenizarlo. Cada modelo tokeniza de una manera distinta, por lo que es necesario indicar el modelo para tokenizar el texto. En nuestro caso vamos a utilizar un modelo llamado [mbart](https://huggingface.co/facebook/mbart-large-50).

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

Definimos una función para tokenizar el texto. Notar que para otros datasets
será necesario cambiar el valor de "gendered" y "neutral" por las columnas que queramos
tokenizar, el resto del código no hará falta tocarlo.

In [ ]:
max_length = 128


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["gendered"],
        max_length=max_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["neutral"], max_length=max_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Tokenizamos el dataset y lo mostramos.

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3513 [00:00<?, ? examples/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['gendered', 'neutral', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3513
    })
    test: Dataset({
        features: ['gendered', 'neutral', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 96
    })
})

Al igual que ocurría con los modelos de clasificación de texto, podemos ver que han aparecido tres nuevas columnas ('input_ids', 'labels' y 'attention_mask') que serán utilizadas para entrenar el modelo. Notar que ya disponemos de una columna labels que será la que tendrá que predecir el modelo.  

Por último, antes de definir nuestro modelo tenemos que definir una función que se va a encargar de preparar los datos para que sean procesados de manera eficiente por el modelo.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer)

## Modelo

Pasamos ahora a definir el modelo, lo primero que vamos a definir son los argumentos con los que vamos a entrenar nuestro modelo.

In [ ]:
from transformers import Seq2SeqTrainingArguments


args = Seq2SeqTrainingArguments(
    # Carpeta donde se guardará el modelo
    output_dir="mbart-neutralization",
    # Evaluar el modelo cada época
    eval_strategy="epoch",
    # Valores para modificar los pesos del modelo
    learning_rate=5.6e-5,
    weight_decay=0.01,
    # Número de ejemplos que se muestran cada vez al modelo
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # Número de veces que que se muestran todos los datos al modelo
    num_train_epochs=2,
    # Realizar predicciones al validar
    predict_with_generate=True,
    push_to_hub=True,
    report_to="none"
)

A continuación definimos nuestro modelo, para ello usamos la clase `AutoModelForSeq2SeqLM` y vamos a utilizar un modelo pre-entrenado (recordar lo que era el transfer learning). Para ello solo tenemos que indicar el nombre de nuestro modelo (definido previamente en la variable `model_checkpoint`.  

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Ahora definimos la función que usaremos para calcular la precisión de nuestro modelo. En este caso usaremos la métrica [sacrebleu](https://huggingface.co/spaces/evaluate-metric/sacrebleu) que nos proporciona la métrica bleu utilizada en el Workshop on Machine Translation (WMT).

In [ ]:
import numpy as np

import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


Ya podemos definir nuestro objeto `trainer` que usaremos para entrenar nuestro modelo (en este caso será un modelo secuencia a secuencia). La estructura de este objeto será siempre la misma. Le tenemos que proporcionar:
1. El modelo.
2. La configuración del entrenamiento.
3. El conjunto de entrenamiento.
4. El conjunto de test.
5. El objeto que prepara los datos.
6. El tokenizador.
7. La métrica.

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-15-7e05dcb5bafc>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Y ahora entrenamos el modelo mediante el método `train`. Este proceso es algo más lento que el entrenamiento de los modelos de clasificación y puede llevar unos 10 minutos.

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,0.018837,98.468500,18.562500
2,0.226700,0.012240,98.551700,18.625000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=880, training_loss=0.1393228671767495, metrics={'train_runtime': 772.5604, 'train_samples_per_second': 9.094, 'train_steps_per_second': 1.139, 'total_flos': 519539733135360.0, 'train_loss': 0.1393228671767495, 'epoch': 2.0})

## Compartiendo el modelo

Una vez que tenemos entrenado nuestro modelo, nos interesa compartirlo con el resto del mundo para que puedan usarlo y también compararlo con otros modelos.

Es por ello que vamos a subir nuestro modelo al hub de huggingface. Para ello tenemos que ejecutar el siguiente comando.

In [ ]:
%cd mbart-neutralization
trainer.push_to_hub(commit_message="Training complete", tags="simplification")

/content/mbart-neutralization


CommitInfo(commit_url='https://huggingface.co/jumava/mbart-neutralization/commit/6c8be396a729dfa99e98eefcba3179367c8c0102', commit_message='Training complete', commit_description='', oid='6c8be396a729dfa99e98eefcba3179367c8c0102', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jumava/mbart-neutralization', endpoint='https://huggingface.co', repo_type='model', repo_id='jumava/mbart-neutralization'), pr_revision=None, pr_num=None)

Al terminar de ejecutarse el comando anterior tendremos nuestro modelo disponible en https://huggingface.co/jumava/mbart-neutralization.
